In [2]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [226]:
sql = "SELECT * \
from ds_qf60"
print(sql)
df = pd.read_sql_query(sql, conn, index_col=None)

df.shape

SELECT * from ds_qf60


(35170, 46)

In [227]:
# df.info()

In [199]:
col_list_ds = ['symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'sz_jt_10','class0' ]
df.tail(1)[col_list_ds]

,symbol,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60,sz_max_5,sz_max_10,sz_max_20,sz_jt_10,class0
35169,689009,2021-04-19,3.04,-0.461137,-1.14476,1.36724,39,0.0,-0.143429,0.522034,...,0,1.30432,1.13952,0.745176,0.524615,0.379001,4.43571,4.43571,0,0


In [230]:
df['sz_jt_10'].value_counts()

0    22561
1     8740
2     3869
Name: sz_jt_10, dtype: int64

In [202]:
# 开发测试：条件取数
# sum = len(df)

# cond2a = df['sz_jt_10']>=2
# df[ cond2a ]['sz_jt_10'].value_counts()

# cond2b = df['sz_jt_10']>1 
# cond2c = df['sz_jt_10']<=2
# cond2bc2 = ( cond2b & cond2c )
# df[ cond2bc2 ]['sz_jt_10'].value_counts()

2    3869
Name: sz_jt_10, dtype: int64

In [239]:
man = pd.DataFrame()
# col = ['pct_change',
#     'diff', 'dea',\
#     'jx_days_ud60',\
#     'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
#     'jx_zs_5', 'jx_zs_10', 'jx_zs_20', \
#     'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
#     'sz_max_10' ]
col = ['pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'rate' ]
man[col] = 0
steps = 30
man

,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,rate


In [240]:
# jx_dg_5 预处理
min_dg_5, max_dg_5 = df['jx_dg_5'].min(), df['jx_dg_5'].max()
step_dg_5 = (max_dg_5 - min_dg_5)/steps
from_dg_5 = min_dg_5
min_dg_5, max_dg_5, step_dg_5, from_dg_5  

(-1.85673, 6.21103, 0.26892533333333335, -1.85673)

In [241]:
# fl_1_3 处理
min_fl_1_3, max_fl_1_3 = df['pct_fl_1_3'].min(), df['pct_fl_1_3'].max()
step_fl_1_3 = (max_fl_1_3 - min_fl_1_3)/steps
from_fl_1_3 = min_fl_1_3
min_fl_1_3, max_fl_1_3, from_fl_1_3, step_fl_1_3 

(1.15793, 66.3237, 1.15793, 2.1721923333333337)

In [170]:
# # df['sz_jt_10'].value_counts()
# test1 = df[df['sz_jt_10'] >=3]['sz_jt_10'].value_counts()
# # len( df[df['sz_jt_10'] >=1]['sz_jt_10'].value_counts() )
# test1, type(test1)
# len(test1)
# num = 1 if test1.empty else 0

# lambda x
# test1.iloc[0] + test1.iloc[1]

0

In [251]:
%%time
# 初始化数据
man.drop(man.index, inplace=True)
from_dg_5 = min_dg_5
from_fl_1_3 = min_fl_1_3

cond_last = df['sz_jt_10'] >=2

# 开始epoch

for i in range(steps):
    to_fl_1_3 = min_fl_1_3 + (i+1)*step_fl_1_3
    print('%.0f : , %.4f,  %.4f'%(i, from_fl_1_3, to_fl_1_3), end=' ' )
    # 处理条件
    # cond_fl_1_3from = df['pct_fl_1_3'] >= from_fl_1_3
    # cond_fl_1_3to = df['pct_fl_1_3'] < to_fl_1_3

    cond_fl_1_3 = (df['pct_fl_1_3'] >= from_fl_1_3) & (df['pct_fl_1_3'] < to_fl_1_3 )
    # cond_basic = ( cond_fl_1_3from & cond_fl_1_3to )
    cond_fl_1_3_class = ( cond_fl_1_3 ) & (cond_last)
    
    # 开始计算   
    sum_step_series  = df[ cond_fl_1_3 ]['sz_jt_10'].value_counts() 
    sum_class_series = df[ cond_fl_1_3_class ]['sz_jt_10'].value_counts() 
    # 
    sum_step = sum_step_series.iloc[0] if not sum_step_series.empty else 0
    sum_class = sum_class_series.iloc[0] if not sum_class_series.empty else 0
    sum_class_rate = 0 if sum_class==0 else sum_step/sum_class
    print( i, 'step:  %.0f %.0f %.4f' %(sum_step, sum_class, sum_class_rate) )
    # 记录到man
    new_data = {'pct_fl_1_3': from_fl_1_3, 'rate': sum_class_rate}
    man = man.append(new_data, ignore_index=True)
    
    # 下一个loop
    from_fl_1_3 = from_fl_1_3 + step_fl_1_3


0 : , 1.1579,  3.3301 0 step:  21042 3600 5.8450
1 : , 3.3301,  5.5023 1 step:  1267 220 5.7591
2 : , 5.5023,  7.6745 2 step:  196 38 5.1579
3 : , 7.6745,  9.8467 3 step:  44 6 7.3333
4 : , 9.8467,  12.0189 4 step:  7 2 3.5000
5 : , 12.0189,  14.1911 5 step:  3 1 3.0000
6 : , 14.1911,  16.3633 6 step:  2 0 0.0000
7 : , 16.3633,  18.5355 7 step:  1 1 1.0000
8 : , 18.5355,  20.7077 8 step:  1 0 0.0000
9 : , 20.7077,  22.8799 9 step:  1 0 0.0000
10 : , 22.8799,  25.0520 10 step:  0 0 0.0000
11 : , 25.0520,  27.2242 11 step:  0 0 0.0000
12 : , 27.2242,  29.3964 12 step:  0 0 0.0000
13 : , 29.3964,  31.5686 13 step:  0 0 0.0000
14 : , 31.5686,  33.7408 14 step:  0 0 0.0000
15 : , 33.7408,  35.9130 15 step:  1 0 0.0000
16 : , 35.9130,  38.0852 16 step:  0 0 0.0000
17 : , 38.0852,  40.2574 17 step:  0 0 0.0000
18 : , 40.2574,  42.4296 18 step:  0 0 0.0000
19 : , 42.4296,  44.6018 19 step:  0 0 0.0000
20 : , 44.6018,  46.7740 20 step:  0 0 0.0000
21 : , 46.7740,  48.9462 21 step:  0 0 0.0000
2

<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from panda

26 step:  0 0 0.0000
27 : , 59.8071,  61.9793 27 step:  0 0 0.0000
28 : , 61.9793,  64.1515 28 step:  0 0 0.0000
29 : , 64.1515,  66.3237 29 step:  1 0 0.0000
CPU times: total: 93.8 ms
Wall time: 242 ms


<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [243]:
man

,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,rate
0,1.157930,NaN,NaN,5.845000
1,3.330122,NaN,NaN,5.759091
2,5.502315,NaN,NaN,5.157895
3,7.674507,NaN,NaN,7.333333
4,9.846699,NaN,NaN,3.500000
5,12.018892,NaN,NaN,3.000000
6,14.191084,NaN,NaN,0.000000
7,16.363276,NaN,NaN,1.000000
8,18.535469,NaN,NaN,0.000000
9,20.707661,NaN,NaN,0.000000


In [222]:
man.drop(man.index, inplace=True)

In [221]:
df

,ts_code,trade_date,symbol,pct_change,diff,dea,bar,jx_days_ud60,jx_xl_5,jx_xl_10,...,test5_8,test10_10,test20_10,test20_20,test20_30,sz_jt_10,sz_jt_20,sz_max_5,sz_max_10,sz_max_20


In [112]:
sum_class

Series([], Name: sz_jt_10, dtype: int64)

In [114]:
len(sum_class)

0

In [113]:
sum_class.item()

ValueError: can only convert an array of size 1 to a Python scalar

In [43]:
'www.example.com'.lstrip('w.')

'example.com'